In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'partsearchgetparttypessdd',
                        'mfr': input_.loc[a, 'Manufacturer'],
                        'pg': input_.loc[a, 'Category'],
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(json_data['html_replace_sections']['ddrepl[parttype_][container]'], 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_type = [type_.strip() for type_ in html.xpath('//select[@id="parttype_"]/option/text()')[1:]]
                
                list_type = [re.sub(' +', ' ', type_) for type_ in list_type]
                
                # = = = = = = = = = = = = = = =
                
                list_type_code = [type_code.strip() for type_code in html.xpath('//select[@id="parttype_"]/option/@value')[1:]]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Manufacturer_No.': int(input_.loc[a, 'Manufacturer_No.']),
                                        'Manufacturer': input_.loc[a, 'Manufacturer'],
                                        'Category_No.': int(input_.loc[a, 'Category_No.']),
                                        'Category': input_.loc[a, 'Category'],
                                        'Type_No.': [i+1 for i in range(len(list_type))],
                                        'Type': list_type,
                                        'Type_Code': list_type_code,
                                        'Tab_Url': ['https://www.rockauto.com/en/parts/' + input_.loc[a, 'Manufacturer'].replace('+', '%2B').replace(',', '%2C') + ',Lennon,' + type_code for type_code in list_type_code]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Manufacturer_No.': int(input_.loc[a, 'Manufacturer_No.']),
                                     'Manufacturer': input_.loc[a, 'Manufacturer'],
                                     'Category_No.': int(input_.loc[a, 'Category_No.']),
                                     'Category': input_.loc[a, 'Category']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Manufacturer'] + ' - ' + input_.loc[a, 'Category'] + '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Manufacturer_No.', 'Category_No.', 'Type_No.'], ascending=[True, True, True]).reset_index(drop=True)
output_correct.to_excel('./3.type-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Manufacturer_No.', 'Category_No.'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./type_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2495

[ok] - ACDELCO - Drivetrain
[尝试次数：1] - [剩余数量：2475] - [当前时间：20:00:02]

[ok] - ACDELCO - Belt Drive
[尝试次数：1] - [剩余数量：2474] - [当前时间：20:00:02]

[ok] - 3D - Interior
[尝试次数：1] - [剩余数量：2473] - [当前时间：20:00:02]

[ok] - 3M - Body & Lamp Assembly
[尝试次数：1] - [剩余数量：2472] - [当前时间：20:00:02]

[ok] - ACDELCO - Body & Lamp Assembly
[尝试次数：1] - [剩余数量：2471] - [当前时间：20:00:02]

[ok] - ACDELCO - Electrical-Connector
[尝试次数：1] - [剩余数量：2470] - [当前时间：20:00:02]

[ok] - ACDELCO - Cooling System
[尝试次数：1] - [剩余数量：2469] - [当前时间：20:00:02]

[ok] - ACDELCO - Brake & Wheel Hub
[尝试次数：1] - [剩余数量：2468] - [当前时间：20:00:02]

[ok] - ACDELCO - Electrical
[尝试次数：1] - [剩余数量：2467] - [当前时间：20:00:02]

[ok] - ACCEL - Ignition
[尝试次数：1] - [剩余数量：2466] - [当前时间：20:00:02]

[ok] - ACCEL - Fuel & Air
[尝试次数：1] - [剩余数量：2465] - [当前时间：20:00:02]

[ok] - 3M - Engine
[尝试次数：1] - [剩余数量：2464] - [当前时间：20:00:02]

[ok] - 3M - Garage Equipment
[尝试次数：1] - [剩余数量：2463] - [当前时间：20:00:02]

[ok] - 3M - Fuel & Air
[尝试次数：1] - [剩余数量：2462] - [当前时间：20:00:03]



[ok] - AMS AUTOMOTIVE - Interior
[尝试次数：4] - [剩余数量：2363] - [当前时间：20:00:17]

[ok] - ANCHOR - Transmission-Automatic
[尝试次数：3] - [剩余数量：2362] - [当前时间：20:00:17]

[ok] - 3M - Interior
[尝试次数：1] - [剩余数量：2361] - [当前时间：20:00:17]

[ok] - ANCHOR - Engine
[尝试次数：4] - [剩余数量：2360] - [当前时间：20:00:17]

[ok] - ALTROM - Suspension
[尝试次数：11] - [剩余数量：2359] - [当前时间：20:00:18]

[ok] - ANCHOR - Suspension
[尝试次数：2] - [剩余数量：2358] - [当前时间：20:00:18]

[ok] - ANCHOR - Transmission-Manual
[尝试次数：5] - [剩余数量：2357] - [当前时间：20:00:18]

[ok] - AMERICAN REMANUFACTURERS INC. - Drivetrain
[尝试次数：8] - [剩余数量：2356] - [当前时间：20:00:18]

[ok] - ANCO - Electrical-Connector
[尝试次数：6] - [剩余数量：2355] - [当前时间：20:00:18]

[ok] - AMERICAN REMANUFACTURERS INC. - Steering
[尝试次数：4] - [剩余数量：2354] - [当前时间：20:00:19]

[ok] - ANCO - Wiper & Washer
[尝试次数：5] - [剩余数量：2353] - [当前时间：20:00:19]

[ok] - API - Belt Drive
[尝试次数：2] - [剩余数量：2352] - [当前时间：20:00:19]

[ok] - APEX - Engine
[尝试次数：4] - [剩余数量：2351] - [当前时间：20:00:19]

[ok] - APEX - Exhaust & Emission
[尝试次数：2

[ok] - AUTO-TUNE - Electrical-Switch & Relay
[尝试次数：4] - [剩余数量：2250] - [当前时间：20:00:42]

[ok] - AUTO EXTRA - Transmission-Manual
[尝试次数：9] - [剩余数量：2249] - [当前时间：20:00:42]

[ok] - AUTO-TUNE - Transmission-Automatic
[尝试次数：1] - [剩余数量：2248] - [当前时间：20:00:42]

[ok] - AUTO EXTRA - Engine
[尝试次数：11] - [剩余数量：2247] - [当前时间：20:00:42]

[ok] - AUTO EXTRA/CHASSIS RITE - Suspension
[尝试次数：3] - [剩余数量：2246] - [当前时间：20:00:42]

[ok] - AUTOLINE - Ignition
[尝试次数：1] - [剩余数量：2245] - [当前时间：20:00:43]

[ok] - AUTOLINE - Brake & Wheel Hub
[尝试次数：1] - [剩余数量：2244] - [当前时间：20:00:44]

[ok] - AUTO EXTRA - Drivetrain
[尝试次数：3] - [剩余数量：2243] - [当前时间：20:00:44]

[ok] - AUTO-TUNE - Engine
[尝试次数：11] - [剩余数量：2242] - [当前时间：20:00:44]

[ok] - ACDELCO - Fuel & Air
[尝试次数：14] - [剩余数量：2241] - [当前时间：20:00:44]

[ok] - AUTODRIVE - Suspension
[尝试次数：2] - [剩余数量：2240] - [当前时间：20:00:44]

[ok] - AUTOLITE - Engine
[尝试次数：2] - [剩余数量：2239] - [当前时间：20:00:44]

[ok] - AUTO 7 - Transmission-Automatic
[尝试次数：10] - [剩余数量：2238] - [当前时间：20:00:44]

[ok] - AUT

[ok] - BECK/ARNLEY - Transmission-Manual
[尝试次数：3] - [剩余数量：2143] - [当前时间：20:01:06]

[ok] - BCA - Drivetrain
[尝试次数：16] - [剩余数量：2142] - [当前时间：20:01:06]

[ok] - BECK/ARNLEY - Steering
[尝试次数：3] - [剩余数量：2141] - [当前时间：20:01:06]

[ok] - BECK/ARNLEY - Body & Lamp Assembly
[尝试次数：11] - [剩余数量：2140] - [当前时间：20:01:06]

[ok] - BECK/ARNLEY - Transmission-Automatic
[尝试次数：2] - [剩余数量：2139] - [当前时间：20:01:06]

[ok] - AUVECO - Wheel
[尝试次数：3] - [剩余数量：2138] - [当前时间：20:01:06]

[ok] - AUTOTECNICA - Exhaust & Emission
[尝试次数：13] - [剩余数量：2137] - [当前时间：20:01:06]

[ok] - BELDEN - Ignition
[尝试次数：7] - [剩余数量：2136] - [当前时间：20:01:07]

[ok] - BENDIX - Wheel
[尝试次数：3] - [剩余数量：2135] - [当前时间：20:01:07]

[ok] - BENDIX - Transmission-Manual
[尝试次数：2] - [剩余数量：2134] - [当前时间：20:01:08]

[ok] - BILSTEIN - Suspension
[尝试次数：3] - [剩余数量：2133] - [当前时间：20:01:08]

[ok] - BOGE - Steering
[尝试次数：4] - [剩余数量：2132] - [当前时间：20:01:09]

[ok] - BILSTEIN - Engine
[尝试次数：4] - [剩余数量：2131] - [当前时间：20:01:09]

[ok] - BECK/ARNLEY - Ignition
[尝试次数：11] - [剩余数量：

[ok] - AUTO-TUNE - Exhaust & Emission
[尝试次数：37] - [剩余数量：2034] - [当前时间：20:01:30]

[ok] - CAMBIARE - Cooling System
[尝试次数：6] - [剩余数量：2033] - [当前时间：20:01:30]

[ok] - BSE - Brake & Wheel Hub
[尝试次数：7] - [剩余数量：2032] - [当前时间：20:01:30]

[ok] - CALORSTAT - Transmission-Automatic
[尝试次数：1] - [剩余数量：2031] - [当前时间：20:01:30]

[ok] - CARDONE - Electrical
[尝试次数：2] - [剩余数量：2030] - [当前时间：20:01:30]

[ok] - CARDONE - Brake & Wheel Hub
[尝试次数：6] - [剩余数量：2029] - [当前时间：20:01:31]

[ok] - CADNA - Heat & Air Conditioning
[尝试次数：14] - [剩余数量：2028] - [当前时间：20:01:31]

[ok] - CARDONE - Steering
[尝试次数：1] - [剩余数量：2027] - [当前时间：20:01:32]

[ok] - CARDONE - Exhaust & Emission
[尝试次数：4] - [剩余数量：2026] - [当前时间：20:01:32]

[ok] - CARDONE - Fuel & Air
[尝试次数：5] - [剩余数量：2025] - [当前时间：20:01:32]

[ok] - CADNA - Engine
[尝试次数：10] - [剩余数量：2024] - [当前时间：20:01:32]

[ok] - CARDONE - Transmission-Automatic
[尝试次数：1] - [剩余数量：2023] - [当前时间：20:01:32]

[ok] - CARDONE - Cooling System
[尝试次数：7] - [剩余数量：2022] - [当前时间：20:01:33]

[ok] - CARDONE - Engi

...

[ok] - SOUTHERN TRUCK - Wheel
[尝试次数：6] - [剩余数量：492] - [当前时间：20:07:07]

[ok] - S.U.R.&R. - Engine
[尝试次数：7] - [剩余数量：491] - [当前时间：20:07:08]

[ok] - SPECIALTY PRODUCTS - Suspension
[尝试次数：5] - [剩余数量：490] - [当前时间：20:07:08]

[ok] - SKP - Heat & Air Conditioning
[尝试次数：2] - [剩余数量：489] - [当前时间：20:07:08]

[ok] - SPARTA - Fuel & Air
[尝试次数：5] - [剩余数量：488] - [当前时间：20:07:08]

[ok] - SOUTHERN TRUCK - Suspension
[尝试次数：8] - [剩余数量：487] - [当前时间：20:07:08]

[ok] - SPECTRA PREMIUM - Body & Lamp Assembly
[尝试次数：5] - [剩余数量：486] - [当前时间：20:07:08]

[ok] - SPECTRE PERFORMANCE - Drivetrain
[尝试次数：2] - [剩余数量：485] - [当前时间：20:07:08]

[ok] - SEAFOAM - Garage Equipment
[尝试次数：24] - [剩余数量：484] - [当前时间：20:07:09]

[ok] - SOUTHERN TRUCK - Fuel & Air
[尝试次数：11] - [剩余数量：483] - [当前时间：20:07:09]

[ok] - SPECTRE PERFORMANCE - Electrical
[尝试次数：3] - [剩余数量：482] - [当前时间：20:07:09]

[ok] - SPECTRA PREMIUM - Engine
[尝试次数：4] - [剩余数量：481] - [当前时间：20:07:09]

[ok] - SPECTRE PERFORMANCE - Engine
[尝试次数：3] - [剩余数量：480] - [当前时间：20:07:10]

[ok] - S

[ok] - SUBARU - Belt Drive
[尝试次数：25] - [剩余数量：386] - [当前时间：20:07:31]

[ok] - SUBARU - Transmission-Manual
[尝试次数：16] - [剩余数量：385] - [当前时间：20:07:31]

[ok] - TIMKEN - Brake & Wheel Hub
[尝试次数：1] - [剩余数量：384] - [当前时间：20:07:31]

[ok] - TIMKEN - Drivetrain
[尝试次数：2] - [剩余数量：383] - [当前时间：20:07:31]

[ok] - TEXACO/HAVOLINE - Fuel & Air
[尝试次数：3] - [剩余数量：382] - [当前时间：20:07:31]

[ok] - SYLVANIA - Electrical-Bulb & Socket
[尝试次数：2] - [剩余数量：381] - [当前时间：20:07:31]

[ok] - TAILORED AUDIO - Interior
[尝试次数：10] - [剩余数量：380] - [当前时间：20:07:32]

[ok] - SUNSONG - Transmission-Automatic
[尝试次数：16] - [剩余数量：379] - [当前时间：20:07:32]

[ok] - TIMKEN - Electrical-Connector
[尝试次数：3] - [剩余数量：378] - [当前时间：20:07:32]

[ok] - TIMKEN - Steering
[尝试次数：2] - [剩余数量：377] - [当前时间：20:07:32]

[ok] - TIMKEN - Engine
[尝试次数：3] - [剩余数量：376] - [当前时间：20:07:32]

[ok] - TAILORED TRUCK COVERS - Body & Lamp Assembly
[尝试次数：12] - [剩余数量：375] - [当前时间：20:07:33]

[ok] - TOMCO, INC. - Exhaust & Emission
[尝试次数：2] - [剩余数量：374] - [当前时间：20:07:33]

[ok] - TI

[ok] - ULTRA-POWER - Suspension
[尝试次数：6] - [剩余数量：273] - [当前时间：20:07:56]

[ok] - ULTRA-POWER - Transmission-Automatic
[尝试次数：1] - [剩余数量：272] - [当前时间：20:07:56]

[ok] - UNITED MOTOR PRODUCTS - Brake & Wheel Hub
[尝试次数：2] - [剩余数量：271] - [当前时间：20:07:57]

[ok] - UNITED MOTOR PRODUCTS - Electrical-Switch & Relay
[尝试次数：3] - [剩余数量：270] - [当前时间：20:07:57]

[ok] - UNDERCAR EXPRESS - Brake & Wheel Hub
[尝试次数：7] - [剩余数量：269] - [当前时间：20:07:57]

[ok] - ULTRA-POWER - Transmission-Manual
[尝试次数：9] - [剩余数量：268] - [当前时间：20:07:57]

[ok] - ÜRO PARTS - Belt Drive
[尝试次数：1] - [剩余数量：267] - [当前时间：20:07:57]

[ok] - UNITED MOTOR PRODUCTS - Electrical-Connector
[尝试次数：5] - [剩余数量：266] - [当前时间：20:07:57]

[ok] - UNITED MOTOR PRODUCTS - Fuel & Air
[尝试次数：2] - [剩余数量：265] - [当前时间：20:07:57]

[ok] - UNITED MOTOR PRODUCTS - Electrical-Bulb & Socket
[尝试次数：3] - [剩余数量：264] - [当前时间：20:07:57]

[ok] - UNITED MOTOR PRODUCTS - Engine
[尝试次数：2] - [剩余数量：263] - [当前时间：20:07:58]

[ok] - UNITED MOTOR PRODUCTS - Ignition
[尝试次数：4] - [剩余数量：262] - 

[ok] - VDO / CONTINENTAL - Body & Lamp Assembly
[尝试次数：5] - [剩余数量：165] - [当前时间：20:08:19]

[ok] - VDO / CONTINENTAL - Brake & Wheel Hub
[尝试次数：3] - [剩余数量：164] - [当前时间：20:08:19]

[ok] - VDO / CONTINENTAL - Engine
[尝试次数：3] - [剩余数量：163] - [当前时间：20:08:19]

[ok] - VDO / CONTINENTAL - Cooling System
[尝试次数：4] - [剩余数量：162] - [当前时间：20:08:19]

[ok] - VALEO - Cooling System
[尝试次数：5] - [剩余数量：161] - [当前时间：20:08:20]

[ok] - VDO / CONTINENTAL - Fuel & Air
[尝试次数：4] - [剩余数量：160] - [当前时间：20:08:20]

[ok] - VDO / CONTINENTAL - Exhaust & Emission
[尝试次数：3] - [剩余数量：159] - [当前时间：20:08:20]

[ok] - VDO / CONTINENTAL - Heat & Air Conditioning
[尝试次数：1] - [剩余数量：158] - [当前时间：20:08:20]

[ok] - VDO / CONTINENTAL - Electrical-Switch & Relay
[尝试次数：1] - [剩余数量：157] - [当前时间：20:08:20]

[ok] - VARIOUS MFR - Electrical
[尝试次数：19] - [剩余数量：156] - [当前时间：20:08:21]

[ok] - VDO / CONTINENTAL - Ignition
[尝试次数：4] - [剩余数量：155] - [当前时间：20:08:21]

[ok] - VARIOUS MFR - Transmission-Automatic
[尝试次数：13] - [剩余数量：154] - [当前时间：20:08:21]

[ok] - 

[ok] - WALKER PRODUCTS - Electrical-Switch & Relay
[尝试次数：3] - [剩余数量：55] - [当前时间：20:08:43]

[ok] - WESTAR - Electrical-Switch & Relay
[尝试次数：1] - [剩余数量：54] - [当前时间：20:08:44]

[ok] - WEATHERTECH - Body & Lamp Assembly
[尝试次数：5] - [剩余数量：53] - [当前时间：20:08:44]

[ok] - WALKER PRODUCTS - Wheel
[尝试次数：7] - [剩余数量：52] - [当前时间：20:08:44]

[ok] - WESTAR - Transmission-Automatic
[尝试次数：1] - [剩余数量：51] - [当前时间：20:08:44]

[ok] - WALKER PRODUCTS - Fuel & Air
[尝试次数：11] - [剩余数量：50] - [当前时间：20:08:44]

[ok] - WALKER PRODUCTS - Transmission-Automatic
[尝试次数：1] - [剩余数量：49] - [当前时间：20:08:45]

[ok] - WESTIN - Electrical-Bulb & Socket
[尝试次数：1] - [剩余数量：48] - [当前时间：20:08:45]

[ok] - WESTIN - Electrical-Connector
[尝试次数：1] - [剩余数量：47] - [当前时间：20:08:45]

[ok] - WESTAR - Body & Lamp Assembly
[尝试次数：6] - [剩余数量：46] - [当前时间：20:08:45]

[ok] - WESTIN - Interior
[尝试次数：2] - [剩余数量：45] - [当前时间：20:08:45]

[ok] - WESTAR - Transmission-Manual
[尝试次数：1] - [剩余数量：44] - [当前时间：20:08:45]

[ok] - WESTIN - Body & Lamp Assembly
[尝试次数：4] - [剩余数量：